# Port

> TODO fill in description

In [1]:
#| default_exp ports

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Callable, Any
import inspect
from collections import deque

import fbdev
from fbdev import Packet
from fbdev.packet_container import PacketContainer, SinglePacketContainer, DequePacketContainer

In [5]:
#|export
__all__ = [
    'BasePort',
    'BaseInputPort', 'SingleInputPort', 'DequeInputPort',
    'BaseOutputPort', 'SingleOutputPort', 'DequeOutputPort'
]

In [6]:
#|hide
show_doc(fbdev.BasePort)

---

### BasePort

>      BasePort (name:str, dtype:type, data_validator:Callable[[Any],bool])

*Initialize self.  See help(type(self)) for accurate signature.*

In [7]:
#|export
class BasePort:
    def __init__(self,
                 name:str,
                 dtype:type,
                 data_validator:Callable[[Any], bool]):
        self.name = name
        self.idx = None
        self._component_process = None
        self.dtype = dtype
        self.data_validator = data_validator
        
    def attach(self, component_process, port_index:int):
        if self._component_process is not None:
            #TODO proper exception in BasePort.attach
            raise Exception("Port is already attached.")
        self._component_process = component_process
        self.idx = port_index
        
    def _check_packet_data_dtype(self,
                                 packet_data):
        if self.dtype is not None:
            if type(packet_data) != self.dtype:
                #TODO logging in BasePort._check_packet_data_dtype
                #TODO proper exceptions in BasePort._check_packet_data_dtype
                #TODO unit test BasePort._check_packet_data_dtype
                raise Exception("Wrong dtype.")
        
    def _check_packet_dtype(self,
                            packet: Type[Packet]):
        if self.dtype is not None:
            if packet.dtype != self.dtype:
                #TODO logging in BasePort._check_packet_dtype
                #TODO proper exceptions in BasePort._check_packet_dtype
                #TODO unit test BasePort._check_packet_dtype
                raise Exception("Wrong dtype.")
            
    def _validate_packet_data(self,
                       packet_data):
        if self.data_validator is not None:
            if not self.data_validator(packet_data):
                #TODO logging in BasePort._validate_packet_data
                #TODO proper exceptions in BasePort._validate_packet_data
                #TODO unit test BasePort._validate_packet_data
                raise Exception("Data is invalid.")

## Input Ports

In [8]:
#|hide
show_doc(fbdev.BaseInputPort)

---

### BaseInputPort

>      BaseInputPort (name:str, dtype:type=None,
>                     data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [9]:
#|export
class BaseInputPort(BasePort, PacketContainer):
    def __init__(self,
                 name:str,
                 dtype: type=None,
                 data_validator: Callable[[Any], bool]=None):
        PacketContainer.__init__(self)
        BasePort.__init__(self, name, dtype, data_validator)
        
    def unload(self):
        raise NotImplementedError("`unload` is disabled for input ports.")
    
    @abstractmethod
    async def receive(self):
        raise NotImplementedError()

In [10]:
#|hide
show_doc(fbdev.SingleInputPort)

---

### SingleInputPort

>      SingleInputPort (name:str, dtype:type=None,
>                       data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [11]:
#|export
class SingleInputPort(BaseInputPort, SinglePacketContainer):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        BaseInputPort.__init__(self, name, dtype, data_validator)
        SinglePacketContainer.__init__(self)

    def load(self, packet):
        self._check_packet_dtype(packet)
        super().load(packet)
        
    async def receive(self):
        await self.empty.wait(False)
        packet = SinglePacketContainer.unload(self)
        packet_data = packet.get_data()
        if inspect.isawaitable(packet_data): packet_data = await packet_data
        self._validate_packet_data(packet_data)
        self._packet = None
        self._set_empty_state()
        return packet_data

In [12]:
port = SingleInputPort("INP1")
packet = Packet("data")
port.load(packet)
assert port.full.get()
data = await port.receive()

In [13]:
#|hide
show_doc(fbdev.DequeInputPort)

---

### DequeInputPort

>      DequeInputPort (name:str, maxsize:int, dtype:type=None,
>                      data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [14]:
#|export
class DequeInputPort(BaseInputPort, DequePacketContainer):
    def __init__(self,
                 name:str,
                 maxsize:int=None,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None,
                 ):
        BaseInputPort.__init__(self, name, dtype, data_validator)
        DequePacketContainer.__init__(self, maxsize)

    def load(self, packet):
        self._check_packet_dtype(packet)
        super().load(packet)
        
    def load_left(self, packet):
        self._check_packet_dtype(packet)
        super().load_left(packet)
    
    async def receive_all(self):
        await self.empty.wait(False)
        packets = DequePacketContainer.unload_all(self)
        packet_datas = [await packet.get_data() for packet in packets]
        for packet_data in packet_datas:
            self._validate_packet_data(packet_data)
        self._empty_ports += list(self._filled_ports)
        self._filled_ports.clear()
        self._update_status_events()
        return packet_datas
        
    async def receive(self):
        await self.empty.wait(False)
        packet = DequePacketContainer.unload(self)
        packet_data = packet.get_data()
        if inspect.isawaitable(packet_data): packet_data = await packet_data
        self._validate_packet_data(packet_data)
        return packet_data
    
    async def receive_right(self):
        await self.empty.wait(False)
        packet = DequePacketContainer.unload(self)
        packet_data = packet.get_data()
        if inspect.isawaitable(packet_data): packet_data = await packet_data
        self._validate_packet_data(packet_data)
        return packet_data

In [15]:
port = DequeInputPort("INP1", maxsize=2)
port.load(Packet("data1"))
port.load(Packet("data2"))
assert port.full.get()
data = await port.receive()
assert data == 'data1'

## Output Ports

In [16]:
#|hide
show_doc(fbdev.BaseOutputPort)

---

### BaseOutputPort

>      BaseOutputPort (name:str, dtype:type=None,
>                      data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [17]:
#|export
class BaseOutputPort(BasePort, PacketContainer):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        super().__init__(name, dtype, data_validator)

    def load(self, packet):
        raise NotImplementedError("`load` is disabled for output ports.")
    
    @abstractmethod
    async def put(self,
            packet_data):
        raise NotImplementedError()

In [18]:
#|hide
show_doc(fbdev.SingleOutputPort)

---

### SingleOutputPort

>      SingleOutputPort (name:str, dtype:type=None,
>                        data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [19]:
#|export
class SingleOutputPort(BaseOutputPort, SinglePacketContainer):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        BaseOutputPort.__init__(self, name, dtype, data_validator)
        SinglePacketContainer.__init__(self)
    
    async def put(self, packet_data):
        await self.full.wait(False)
        packet = Packet(packet_data)
        self._validate_packet_data(packet_data)
        self._check_packet_data_dtype(packet_data)
        SinglePacketContainer.load(self, packet)

In [20]:
port = SingleOutputPort("OUT")
await port.put("data")
assert port.unload().get_data() == "data"

In [21]:
#|hide
show_doc(fbdev.DequeOutputPort)

---

### DequeOutputPort

>      DequeOutputPort (name:str, maxsize:int, dtype:type=None,
>                       data_validator:Callable[[Any],bool]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [22]:
#|export
class DequeOutputPort(BaseOutputPort, DequePacketContainer):
    def __init__(self,
                 name:str,
                 maxsize:int=None,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None,
                 ):
        BaseOutputPort.__init__(self, name, dtype, data_validator)
        DequePacketContainer.__init__(self, maxsize)

    def load_right(self):
        raise NotImplementedError("`load_right` is disabled for output ports.")
    
    async def put(self, packet_data):
        await self.full.wait(False)
        packet = Packet(packet_data)
        self._validate_packet_data(packet_data)
        self._check_packet_data_dtype(packet_data)
        DequePacketContainer.load(self, packet)
        
    async def put_left(self, packet_data):
        await self.full.wait(False)
        packet = Packet(packet_data)
        self._validate_packet_data(packet_data)
        self._check_packet_data_dtype(packet_data)
        DequePacketContainer.load_left(self, packet)

In [23]:
port = DequeOutputPort("OUT", maxsize=2)
await port.put("data1")
await port.put("data2")
assert port.unload().get_data() == "data1"
assert port.unload().get_data() == "data2"